# Python functies voor Odata4
Functies voor ophalen, inspecteren en samenvoegen van data van CBS

## Meer info

- https://www.cbs.nl/nl-nl/onze-diensten/open-data/open-data-v4/snelstartgids-odata-v4
- https://www.cbs.nl/nl-nl/onze-diensten/open-data/open-data-v4/metadata-odata-v4
- Ook code voor R beschikbaar.

# Definieren functies

In [1]:
import pandas as pd
import requests
import re
import time

In [2]:
import pandas as pd
import requests

def get_odata(target_url):
    
    """"
    De functie gebruikt een API genaamd OData om data van het CBS op te halen.
    De data wordt in stukken opgehaald en in een pandas dataframe gezet.
    De URL moet er zo uitzien: "https://odata4.cbs.nl/CBS/83765NED"
    De code van de tabel die je zoekt vindt je via Statline.
    Ga naar de data in Statline op de website van het CBS en kijk naar de URL om de code te vinden.
    """
    
    data = pd.DataFrame()
    while target_url:
        r = requests.get(target_url).json()
        data = data.append(pd.DataFrame(r['value']))
        
        if '@odata.nextLink' in r:
            target_url = r['@odata.nextLink']
        else:
            target_url = None
            
    return data

De functie gebruikt een API genaamd OData om data van het CBS op te halen.
De data wordt in stukken opgehaald en in een pandas dataframe gezet.
De URL moet er zo uitzien: "https://odata4.cbs.nl/CBS/83765NED"
De code van de tabel die je zoekt vindt je via Statline.
Ga naar de data in Statline op de website van het CBS en kijk naar de URL om de code te vinden.
Deze API is hierarchisch. 

In [44]:
def get_observations(table_url, url_filter = ""):
    
    """Haal de tabel met metingen op. Filteren om minder of specifiekere data op te vragen is mogelijk as volgt:
    url_filter = ?$filter=WijkenEnBuurten eq 'GM0363' and Measure eq 'T001036'
    Door de filteren op deze kolommen kun je een plaats (land, gemeente, wijk of buurt) en dan een soort meting kiezen.
    De website van het CBS vermeldt niet hoe je op meerdere waarden in één kolom filtert. 
    Die mogelijkheid bestaat waarschijnlijk wel. Je hoeft niet beide kolommen te gebruiken voor het filter.
    """
    
    if url_filter == "":
        target_url = table_url + "/Observations"
    elif "?$filter=" in url_filter:
        target_url = table_url + "/Observations" + url_filter
    else:
        print("WAARSCHUWING! FILTER NIET GOED GEFORMATTEERD. VERGELIJK MET VOORBEELDEN OF GA NAAR ")
        print("\n https://www.cbs.nl/nl-nl/onze-diensten/open-data/open-data-v4/filters-odata-v4")
        print("\n http://docs.oasis-open.org/odata/odata/v4.0/errata03/os/complete/part2-url-conventions/odata-v4.0-errata03-os-part2-url-conventions-complete.html#_Toc453752358")
        print("\n sectie 5.1.1.5")
        #return None
        pass
    
    data = get_odata(target_url)
    
    return(data)

url_voorbeeld = "https://odata4.cbs.nl/CBS/83765NED"

# Wat is de structuur van de data?

In [4]:
data_structuur = get_odata(url_voorbeeld)
print(data_structuur)
data_Mcodes = get_odata(url_voorbeeld+"/MeasureCodes")
print(data_Mcodes.columns)

        kind                   name                    url
0  EntitySet          MeasureGroups          MeasureGroups
1  EntitySet           MeasureCodes           MeasureCodes
2  EntitySet             Dimensions             Dimensions
3  EntitySet  WijkenEnBuurtenGroups  WijkenEnBuurtenGroups
4  EntitySet   WijkenEnBuurtenCodes   WijkenEnBuurtenCodes
5  EntitySet           Observations           Observations
6  Singleton             Properties             Properties
Index(['DataType', 'Decimals', 'Description', 'Format', 'Identifier', 'Index',
       'MeasureGroupId', 'PresentationType', 'Title', 'Unit'],
      dtype='object')


In [5]:
# Maak een csv van de codes om ze in Excel goed te bekijken
data_Mcodes.to_csv("MeasureCodes.csv", sep=";", na_rep="None")
data_Mcodes[['Title','Description','Identifier','MeasureGroupId']].to_csv("features_alles.csv", sep=";", na_rep="None")
data_Mcodes
#data_Mcodes['Title'].values
#data_Mcodes["Identifier"]

,DataType,Decimals,Description,Format,Identifier,Index,MeasureGroupId,PresentationType,Title,Unit
0,Long,0,,None,T001036,7,M000352,Absolute,Aantal inwoners,aantal
1,Long,0,,None,3000,9,T001038,Absolute,Mannen,aantal
2,Long,0,,None,4000,10,T001038,Absolute,Vrouwen,aantal
3,Long,0,Aantal inwoners dat op 1 januari 0 tot 15 jaar...,None,10680,12,10000,Absolute,0 tot 15 jaar,aantal
4,Long,0,Aantal inwoners dat op 1 januari 15 tot 25 jaa...,None,53050,13,10000,Absolute,15 tot 25 jaar,aantal
5,Long,0,Aantal inwoners dat op 1 januari 25 tot 45 jaa...,None,53310,14,10000,Absolute,25 tot 45 jaar,aantal
6,Long,0,Aantal inwoners dat op 1 januari 45 tot 65 jaa...,None,53715,15,10000,Absolute,45 tot 65 jaar,aantal
7,Long,0,Aantal inwoners dat op 1 januari 65 jaar of ou...,None,80200,16,10000,Absolute,65 jaar of ouder,aantal
8,Long,0,Het aantal inwoners dat op 1 januari ongehuwd ...,None,1010,18,T001019,Absolute,Ongehuwd,aantal
9,Long,0,Het aantal inwoners dat op 1 januari gehuwd is...,None,1020,19,T001019,Absolute,Gehuwd,aantal


In [6]:
# het voorbeeld van de website van CBS:
#table_url = "https://odata4.cbs.nl/CBS/83765NED"

#target_url = table_url + "/Observations"
#data = get_odata(target_url)
#print(data.head())

In [7]:
#tabel 2019
#lastyear_url = "https://odata4.cbs.nl/CBS/84583NED"

## Filteren van query
Het filteren van de data maakt het downloaden sneller.
Het filteren van 'Observations' data kan door code van dit format achter de url te plakken:

**?$filter=WijkenEnBuurten eq 'GM0363' and Measure eq 'T001036'**

De code uit de kolom WijkenEnBuurten kun je vinden met 

**get_odata(table_url + "/WijkenEnBuurtenCodes")**

De 'Title' kolom van deze tabel bevat de namen van wijken, zodat je kan zoeken met str.find, <>.str.contains of Regex.
Zoals wel vaken met tektskolommen moet je dan vertrouwen op de volledigheid en consistentie.
Achteraf controleren of je wel de juiste weijken hebt s dus wel aangeraden.
De kolom WijkenenBuurten bevat zowel landen, gemeenten, wijken als buurten.
Aan het voorvoegsel van twee letters kun je zien met welke soort regio je te maken hebt.

Zie https://www.cbs.nl/nl-nl/onze-diensten/open-data/open-data-v4/filters-odata-v4 voor meer uitleg.

In [8]:
table_test = get_observations(url_voorbeeld, url_filter="?$filter=contains(WijkenEnBuurten,'GM')")
#table_test = get_observations(url_voorbeeld)

    Id  Measure    Value ValueAttribute WijkenEnBuurten
0  103  T001036  25286.0           None          GM1680
1  104     3000  12603.0           None          GM1680
2  105     4000  12683.0           None          GM1680
3  106    10680   3572.0           None          GM1680
4  107    53050   2558.0           None          GM1680


In [42]:
print(table_test.columns)
print(table_test.head(5))
# Kolom ValueAttribute heeft geen waarden

Index(['Id', 'Measure', 'Value', 'ValueAttribute', 'WijkenEnBuurten'], dtype='object')
    Id  Measure    Value ValueAttribute WijkenEnBuurten
0  103  T001036  25286.0           None          GM1680
1  104     3000  12603.0           None          GM1680
2  105     4000  12683.0           None          GM1680
3  106    10680   3572.0           None          GM1680
4  107    53050   2558.0           None          GM1680


In [10]:
# Deze tabel bevat alle gemeente, maar GEEN info over wijken en buurten
data_gemeenten = get_odata(url_voorbeeld+"/WijkenEnBuurtenGroups")
print(data_gemeenten.size)
print(data_gemeenten.columns)
print(data_gemeenten.head(10))
# ParentId geeft voor wijken de buurt aan, voor wijken de plaatsnaam, voorplaatsnamen de gemeente etc.

1960
Index(['Description', 'Id', 'Index', 'ParentId', 'Title'], dtype='object')
  Description      Id  Index ParentId                           Title
0        None    WBGM      0     None  Wijken en buurten per gemeente
1        None  GM1680      1     WBGM                     Aa en Hunze
2        None  GM0738      2     WBGM                         Aalburg
3        None  GM0358      3     WBGM                        Aalsmeer
4        None  GM0197      4     WBGM                          Aalten
5        None  GM0059      5     WBGM                   Achtkarspelen
6        None  GM0482      6     WBGM                    Alblasserdam
7        None  GM0613      7     WBGM                   Albrandswaard
8        None  GM0361      8     WBGM                         Alkmaar
9        None  GM0141      9     WBGM                          Almelo


In [11]:
filter_gemeente = data_gemeenten['Title'] == 'Oss'
print(data_gemeenten[filter_gemeente])
selectie_gem = data_gemeenten[filter_gemeente]['Id'].values[0]
selectie_gem

    Description      Id  Index ParentId Title
254        None  GM0828    254     WBGM   Oss


'GM0828'

In [12]:
# Deze tabel bevat codes van wijken en buurten en toont bij welke gemeente ze horen. De gemeenten staan er ook in.
data_geocodes = get_odata(url_voorbeeld+"/WijkenEnBuurtenCodes")
print(data_geocodes.size)
print(data_geocodes.columns)
print(data_geocodes.head(10))
# Ik denk dat DimensionGroupId te maken heeft met hierarchische indeling maar niet hetzelfde is als parentId.
# DetailRegionCode is hetzelfde als Identifier

100002
Index(['Description', 'DetailRegionCode', 'DimensionGroupId', 'Identifier',
       'Index', 'Title'],
      dtype='object')
  Description DetailRegionCode DimensionGroupId  Identifier  Index  \
0                         None               NL        NL00      1   
1                       GM1680               GM      GM1680      2   
2                     WK168000           GM1680    WK168000      3   
3                   BU16800000           GM1680  BU16800000      4   
4                   BU16800009           GM1680  BU16800009      5   
5                     WK168001           GM1680    WK168001      6   
6                   BU16800100           GM1680  BU16800100      7   
7                   BU16800109           GM1680  BU16800109      8   
8                     WK168002           GM1680    WK168002      9   
9                   BU16800200           GM1680  BU16800200     10   

                     Title  
0                Nederland  
1              Aa en Hunze  
2          

In [13]:
# Gebruik van Regular Expressions sterk aanbevolen om verschil met hele woorden te zien,
# Maakt implementatie wel iets ingewikkelder.
filter_BU = data_geocodes['Identifier'].str.contains("BU")
filter_WK = data_geocodes['Identifier'].str.contains("WK")

filter_BUWKopgem = data_geocodes['DimensionGroupId'] == selectie_gem
#print(data_geocodes[filter_BUWKopgem])
selectie_WK = data_geocodes[filter_WK & filter_BUWKopgem]["Identifier"]
print(selectie_WK)
#selectie_BU = data_geocodes[filter_BU & filter_BUWKopgem]["Identifier"]
#print(selectie_BU)

11353    WK082800
11357    WK082801
11365    WK082802
11369    WK082803
11378    WK082804
11384    WK082805
11389    WK082806
11400    WK082807
11411    WK082808
11416    WK082809
11422    WK082810
11428    WK082811
11432    WK082812
11436    WK082813
11440    WK082814
11446    WK082815
11450    WK082816
11454    WK082817
11460    WK082818
11463    WK082819
11468    WK082820
11471    WK082821
11477    WK082822
Name: Identifier, dtype: object


In [14]:
url_test = "?$filter=WijkenEnBuurten eq '"+selectie_gem+"'"
# Alleen op deze manier kan je een ID uit de geografische tabel in het url-filter plaatsen. Ook de spaties tellen.)
#table_test = get_observations(url_voorbeeld, url_filter=url_test)
#table_test.head()

In [54]:
def maak_lijst_plaatsen(database, plaatsnaam, soort, exact = True, is_gemeente=True):
    """Voer de code van de database, de naam van een gemeente in, en GM, WK of BU.
    Functie filtert met URL. Als exact = False, dan tellen zoekresultaten die de string bevatten ook mee.
    Exact filter
    Functie maakt een lijst van codes binnen die gemeente.
    Zoeken op plaats binnen een gemeente kan ook maar is niet robuust want dan worden alle gemeenten doorzocht."""

    url = "https://odata4.cbs.nl/CBS/" + database
    filter_url = "?$filter="
    if is_gemeente:
        data_gemeenten = get_odata(url+"/WijkenEnBuurtenGroups")
        if exact:
            filter_gemeente = data_gemeenten['Title'] == plaatsnaam
        else:
            filter_gemeente = data_gemeenten['Title'].str.contains(plaatsnaam)
            if sum(filter_gemeente) == 1:
                pass
            else:
                print("Kies uit één van deze gemeenten en roep de functie nogmaals aan met de selectie. \n")
                print(data_gemeenten[filter_gemeente])
                return data_gemeenten[filter_gemeente]['Identifier']
        
        selectie_gem = data_gemeenten[filter_gemeente]["Id"].values[0]
        plaatsen = get_odata(url+"/WijkenEnBuurtenCodes"+filter_url+"DimensionGroupId eq '"+selectie_gem+"'")
    
    else:
        if exact:
            plaatsen = get_odata(url+"/WijkenEnBuurtenCodes"+filter_url+"Title eq '"+plaatsnaam+"'")
        else:
            plaatsen = get_odata(url+"/WijkenEnBuurtenCodes"+filter_url+"contains(Title, '"+plaatsnaam+"')")
        
    if soort == "GM":
        return plaatsen[plaatsen["Identifier"].str.contains("GM")]["Identifier"]
    elif soort == "WK":
        return plaatsen[plaatsen["Identifier"].str.contains("WK")]["Identifier"]
    elif soort == "BU":
        return plaatsen[plaatsen["Identifier"].str.contains("BU")]["Identifier"]
    pass

In [60]:
lijst_test = maak_lijst_plaatsen("83765NED", "Oss", "WK")
print(type(lijst_test))
lijst_test

<class 'pandas.core.series.Series'>


0      WK082800
4      WK082801
12     WK082802
16     WK082803
25     WK082804
31     WK082805
36     WK082806
47     WK082807
58     WK082808
63     WK082809
69     WK082810
75     WK082811
79     WK082812
83     WK082813
87     WK082814
93     WK082815
97     WK082816
101    WK082817
107    WK082818
110    WK082819
115    WK082820
118    WK082821
124    WK082822
Name: Identifier, dtype: object

# Samenvoegen data

In [16]:
def formatteer_tabel(df):
    """Formatteert een tabel met observaties naar een formaat met één kolom voor elke meting en één rij per plaats.
    De codes voor plaatsen worden de index, de kolommen krijgen als naam de Identifier van de measure.
    Dat betekent dat extra informatie over de hierarchie van measures of plaatsen nog moet worden toegevoegd.
    Deze functie verwijdert dubbele waarden en vult NaN's niet in."""
    
    df2 = df.pivot(index="Id", columns="Measure", values="Value")
    df2 = df2.merge(df[["Id","WijkenEnBuurten"]], how="left", left_index=True, right_on="Id").drop("Id", axis=1)
    df2 = df2.groupby("WijkenEnBuurten").first()
    return df2

#formatteer_tabel(table_test)

In [17]:
#test2 = table_test.pivot(index="Id", columns="Measure", values="Value") # Zelfs voor hele dataset snel te doen
#test2 = test2.merge(table_test[["Id","WijkenEnBuurten"]], how="left", left_index=True, right_on="Id").drop("Id", axis=1)
#test2.head(20)
#test2.groupby("WijkenEnBuurten").first()
# Deze tabel heeft het gewenste format, maar de tabel heeft een index als lijst van kolommen

table_final = formatteer_tabel(table_test)
table_final.head()


,1010,1014800_1,1014800_2,1014800_3,1014850_2,1014850_3,1014850_4,1016030,1016040,1020,...,ZW10320_2,ZW10340,ZW25805_1,ZW25805_2,ZW25806_1,ZW25806_2,ZW25807,ZW25808,ZW25810_1,ZW25810_2
WijkenEnBuurten,,,,,,,,,,,,,,,,,,,,,
GM0003,5075.0,48.0,2830.0,1630.0,51.0,2140.0,1220.0,1733.0,1800.0,4997.0,...,1960.0,28.0,2560.0,1360.0,2650.0,1550.0,15.0,85.0,1790.0,960.0
GM0005,4599.0,68.0,2830.0,1680.0,31.0,2070.0,1280.0,1580.0,1526.0,4720.0,...,2020.0,14.0,2530.0,1380.0,2520.0,1510.0,14.0,86.0,1690.0,890.0
GM0007,3259.0,71.0,3520.0,2040.0,28.0,2370.0,1620.0,1179.0,1606.0,4294.0,...,2130.0,6.0,2230.0,1470.0,2400.0,1630.0,6.0,94.0,1810.0,1080.0
GM0009,3231.0,77.0,2890.0,1730.0,23.0,2100.0,1340.0,1176.0,987.0,3137.0,...,2000.0,9.0,2530.0,1440.0,2570.0,1610.0,15.0,85.0,1740.0,780.0
GM0010,10264.0,62.0,2890.0,1670.0,37.0,2160.0,1310.0,3585.0,3815.0,10894.0,...,2000.0,22.0,2650.0,1400.0,2660.0,1570.0,9.0,91.0,1750.0,1020.0


# Selecteren features

In [18]:
def feature_select(df, csv_file):
    selectie = set(pd.read_csv(csv_file, sep=";")["Identifier"])
    test = set(df.columns)
    selectie = [feature for feature in selectie if feature in df.columns]
    df = df[selectie]
    return df
    
def generate_feature_dict(csv_file):
    feature_frame = pd.read_csv(csv_file, sep=";")
    feature_dict = {a:b for a,b in zip(feature_frame['Identifier'], feature_frame['Title']) }
    return feature_dict

def wissel_naar_titels(df,feature_dict):
    titels = [feature_dict[Identifier] for Identifier in df.columns]
    df.columns = titels
    
    return df

In [19]:
feature_all = pd.read_csv("features_alles.csv",sep=";")
table_final = feature_select(table_final, "features_test2.csv")

# Het is handiger om pas op het laatste moment de identifiers in de kolomnamen te vervangen door de titels
table_final = wissel_naar_titels(table_final, generate_feature_dict("features_alles.csv"))
table_final.head(15)

,Oppervlakte land,Gemiddeld elektriciteitsverbruik totaal,25 tot 45 jaar,Personen per soort uitkering; Bijstand,Percentage meergezinswoning,Verweduwd,Aantal inkomensontvangers,Eenpersoonshuishoudens,Woningvoorraad,Personenauto's; 6 jaar en ouder,...,Gemiddelde huishoudensgrootte,20% huishoudens met hoogste inkomen,Percentage woningen met stadsverwarming,Sterfte totaal,Hoekwoning,Scholen binnen 3 km,In bezit overige verhuurders,"Vernieling, misdrijf tegen openbare orde",Huishoudens met kinderen,15 tot 25 jaar
WijkenEnBuurten,,,,,,,,,,,,,,,,,,,,,
GM0003,2378.0,2460.0,2528.0,520.0,28.0,875.0,9400.0,2067.0,5899.0,3895.0,...,2.1,11.5,NaN,152.0,1550.0,6.3,7.0,3.0,1733.0,1235.0
GM0005,4454.0,2580.0,2196.0,190.0,14.0,525.0,8100.0,1293.0,4569.0,3560.0,...,2.3,17.8,NaN,103.0,1510.0,3.8,8.0,2.0,1580.0,1206.0
GM0007,10837.0,3170.0,1596.0,260.0,6.0,636.0,7100.0,1204.0,4117.0,3965.0,...,2.2,13.6,NaN,136.0,1630.0,1.9,9.0,3.0,1179.0,845.0
GM0009,4531.0,2740.0,1524.0,100.0,9.0,457.0,5500.0,795.0,3045.0,2660.0,...,2.4,19.6,NaN,84.0,1610.0,2.5,5.0,3.0,1176.0,829.0
GM0010,13313.0,2630.0,5014.0,1110.0,22.0,1786.0,19700.0,4127.0,12237.0,8675.0,...,2.1,12.5,NaN,309.0,1570.0,6.7,9.0,5.0,3585.0,2611.0
GM0014,9534.0,2320.0,61584.0,11170.0,64.0,6656.0,161700.0,75299.0,101169.0,46325.0,...,1.6,10.1,NaN,1397.0,1530.0,10.6,25.0,8.0,21628.0,48066.0
GM0015,8671.0,3130.0,2669.0,190.0,8.0,581.0,9300.0,1261.0,4982.0,5235.0,...,2.5,18.3,NaN,98.0,1480.0,3.1,7.0,3.0,2032.0,1517.0
GM0017,4542.0,3020.0,3587.0,310.0,23.0,1346.0,14900.0,3133.0,8738.0,6800.0,...,2.2,30.5,NaN,269.0,1680.0,6.3,17.0,4.0,2821.0,1893.0
GM0018,6646.0,2730.0,7668.0,1460.0,30.0,2235.0,26700.0,5780.0,16015.0,12240.0,...,2.1,11.5,NaN,426.0,1600.0,8.7,9.0,5.0,5094.0,3738.0


# To do: 

- Test manieren om data op te vragen op basis van adressen en een lijst features
- Fix Odata3 parser met slim gekozen filters en dezelfde functies als OData4
- Analyseer en bewerk NaN's. Veel algoritmen van SciKit kunnen niet overweg met NaN.

Dan zouden alle gewenste functies klaar moeten zijn en kan het notebook worden opgeschoont of omgezet naar .py
De hierarchische indeling van locaties moet nog wel toegevoegd worden maar omdat daar andere bestanden voor nodig zijn
is het netjes om in een ander notebook verder te gaan.

In [66]:
def concat_DataFrames(LoF):
    """Neemt een lijst of dict van DataFrames aan uit verschillende zoekopdrachten en maakt er één geheeld van.
    Kolommen moeten wel hetzelfde zijn."""
    
    if type(LoF) == list:
        fullframe = LoF[0]
        for frame in LoF[1:]:
            fullframe = pd.concat([fullframe,frame], axis=0, sort=False)
    elif type(LoF) == dict:
        keys = list(LoF.keys())
        fullframe = LoF[keys[0]]
        for key in keys[1:]:
            fullframe = pd.concat([fullframe,LoF[key]], axis=0, sort=False)
    else:
        pass
    
    return fullframe

In [61]:
# Alles bij elkaar (in één functie):

# Deze functie moet nog uitvoerig getest worden!!!

def parserOData4(database, plaatsen, feature_dict=None, custom_filter = ""):
    """Deze functie neemt een code van de dataset op Odata4 en van plaatsCODES aan.
    Je kan ook "GM", "BU" of "WK" invoeren in plaats van en lijst met codes
    om data uit alle gemeenten, buurten of wijken op te vragen.
    Het vraagt dan per code de dataset op en voegt de dataframes samen.
    De code van de dataset is de combinatie van cijfers en letters achteraan de url:
    https://odata4.cbs.nl/CBS/83765NED
    Je kan ook zelf een filter genereren dat achter de andere filters geplakt wordt.
    Zie docstring van get_observations voor meer info"""
    
    table_url = "https://odata4.cbs.nl/CBS/"+database
    if custom_filter != "":
        custom_filter = " and "+custom_filter
    
    
    if type(plaatsen) == list or type(plaatsen) == pd.Series:
        frames = {}
        for plaats in plaatsen:
            url_filter = "?$filter=WijkenEnBuurten eq '"+plaats+"'"+custom_filter
            frames[plaats] = formatteer_tabel(get_observations(table_url, url_filter))
            time.sleep(2)
        print("Download geslaagd.")
        observations = concat_DataFrames(frames)
        if type(feature_dict) == dict:
            # Geen functie voor nodig, één list comrpehesion is al voldoende
            feature_selectie = [feature for feature in feature_dict.keys() if (feature in observations.columns)]
            observations = observations[feature_selectie]
        return observations
    
    elif plaatsen == "GM":
        url_filter = "?$filter=contains(WijkenEnBuurten,'GM')"
    elif plaatsen == "BU":
        url_filter = "?$filter=contains(WijkenEnBuurten,'BU')"
    elif plaatsen == "WK":
        url_filter = "?$filter=contains(WijkenEnBuurten,'WK')"
    else:
        url_filter = "?$filter=WijkenEnBuurten eq '"+plaatsen+"'"
    
    
    url_filter = url_filter + custom_filter
    print(table_url + url_filter)
    observations = get_observations(table_url, url_filter)
    print("Download geslaagd.")
    
    observations = formatteer_tabel(observations)
    if type(feature_dict) == dict:
        # Geen functie voor nodig, één list comrpehesion is al voldoende
        feature_selectie = [feature for feature in feature_dict.keys() if (feature in observations.columns)]
        observations = observations[feature_selectie]
    
    return observations

In [62]:
#Test met GM
#parserOData4("83765NED", "GM")
#Test met enkele code
#parserOData4("83765NED",selectie_gem)
#Test beide met extra filter
#parserOData4("83765NED", "GM", custom_filter="contains(Measure, 'M000')")
parserOData4("83765NED", selectie_gem, custom_filter="contains(Measure, 'M000')")

https://odata4.cbs.nl/CBS/83765NED?$filter=WijkenEnBuurten eq 'GM0828' and contains(Measure, 'M000')
Download geslaagd.


,M000100,M000114,M000173_1,M000173_2,M000179_1,M000179_2,M000200_2,M000219_2,M000221_2,M000223,M000224,M000226,M000232,M000297,M000302,M000368
WijkenEnBuurten,,,,,,,,,,,,,,,,
GM0828,554.0,2.3,904.0,10.0,705.0,7.0,7755.0,1450.0,3290.0,30.4,24.7,5.7,72500.0,39130.0,59.3,1.2


In [67]:
# Functie die lijsten maakt werkt!
#Test parser met lijst
#maak_lijst_plaatsen("83765NED","Oss", "WK")
#parserOData4("83765NED", maak_lijst_plaatsen("83765NED","Oss", "WK"))
#Test parser met lijst en extra filter
parserOData4("83765NED", maak_lijst_plaatsen("83765NED","Oss", "WK"), custom_filter="contains(Measure,'M000')")

Download geslaagd.


,M000100,M000114,M000173_1,M000173_2,M000179_1,M000179_2,M000200_2,M000219_2,M000221_2,M000223,M000224,M000226,M000232,M000297,M000302,M000368
WijkenEnBuurten,,,,,,,,,,,,,,,,
WK082800,5170.0,1.7,45.0,8.0,115.0,20.0,675.0,1060.0,2330.0,26.8,23.6,9.2,4800.0,3306.0,57.7,0.8
WK082801,4702.0,2.1,110.0,11.0,65.0,6.0,480.0,1220.0,2960.0,27.3,21.9,9.2,7700.0,4499.0,57.4,0.9
WK082802,152.0,2.7,0.0,5.0,5.0,10.0,540.0,2400.0,4170.0,32.3,25.1,3.1,300.0,134.0,65.7,NaN
WK082803,2948.0,2.1,65.0,10.0,40.0,6.0,415.0,1480.0,2990.0,33.0,27.3,5.7,5100.0,2922.0,60.8,1.0
WK082804,4106.0,2.5,90.0,11.0,40.0,4.0,460.0,1330.0,3470.0,31.3,24.5,5.1,6300.0,3213.0,62.2,1.2
WK082805,43.0,2.5,0.0,6.0,0.0,19.0,385.0,2960.0,5620.0,38.1,30.3,NaN,100.0,59.0,71.2,NaN
WK082806,3167.0,2.2,120.0,9.0,145.0,11.0,660.0,1390.0,2960.0,27.2,22.0,7.7,10500.0,5923.0,51.5,1.0
WK082807,4553.0,2.3,100.0,7.0,85.0,6.0,720.0,1370.0,3280.0,31.5,25.8,4.5,11100.0,5786.0,58.6,1.2
WK082808,45.0,3.2,0.0,5.0,0.0,2.0,70.0,2710.0,4790.0,34.4,26.7,3.7,300.0,110.0,67.9,1.5
